In [4]:
!conda install -y -c conda-forge fbprophet
# !pip install mlflow
# !pip install --upgrade pandas
import sagemaker
import pandas as pd
import s3fs
from fbprophet import Prophet
import mlflow
from mlflow.pyfunc import log_model
import mlflow.pyfunc
from fbprophet.diagnostics import cross_validation, performance_metrics
import os

Solving environment: done

# All requested packages already installed.



In [6]:
bucket = sagemaker.Session().default_bucket()
# file_suffix = '_daily_raw_with_3_mo_target.csv'
folder = 'stock_data_raw_with_3_mo_target'
fs = s3fs.S3FileSystem()
files = fs.ls(f's3://{bucket}/{folder}')
try:
    exp_id = mlflow.create_experiment('fbprophet_all_')
except Exception as e:
    display(e)
    exp_id= mlflow.get_experiment_by_name('fbprophet_all_')

mlflow.exceptions.MlflowException("Experiment 'fbprophet_all_' already exists.")

In [ ]:
# for file in files[:1]:

metric = 'low_window_20_shift_0_mean_percent_change_percent_lag_90_shift_-90' ##high
rolling_window=0.1
for file in files:
    with mlflow.start_run(experiment_id=exp_id):
        mlflow.log_param('file', file)
        mlflow.log_param('metric',metric)
    #     mlflow.log_param('rolling_window', rolling_window)
        dft = pd.read_csv(f's3://{file}')#.dropna()
        df = dft[['Unnamed: 0', metric]]
        df.columns = ['ds', 'y']
    #     display(df)
        m = Prophet(growth='linear')
        m.fit(df)
    #     log_model('model', FbProphetWrapper(m))
    #     df_cv = cross_validation(m, horizon = '90 days')
    #     df_p = performance_metrics(df_cv, rolling_window=rolling_window)
    #     mlflow.log_metric("rmse", df_p.loc[0,'RMSE'])
    #     mlflow.log_metric("MSE", df_p.loc[0,'MSE'])
    #     mlflow.log_metric("MAE", df_p.loc[0,'MAE'])
    #     mlflow.log_metric("MAPE", df_p.loc[0,'MAPE'])
    #     mlflow.log_metric("MDAPE", df_p.loc[0,'MDAPE'])
    #     mlflow.log_metric("SMAPE", df_p.loc[0,'SMAPE'])
    #     mlflow.log_metric("COVERAGE", df_p.loc[0,'COVERAGE'])   
        periods=90
        future = m.make_future_dataframe(periods=periods)
        mlflow.log_param('length of predeiction', periods)
    #     display(future)
        forecast = m.predict(future)
        yhat = forecast.loc[len(forecast.index)-1, 'yhat']
        y = df.loc[len(df.index)-1, 'y']
        percent_change = (yhat-y)/y
        mlflow.log_metric('percent change', percent_change)

    #     fig1 = m.plot(forecast)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/opt/conda/lib/python3.7/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/opt/conda/lib/python3.7/site-packages/pystan/misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):


In [23]:
forecast.loc[len(forecast.index)-1, 'yhat']

2.155645709745646

In [8]:
runs = mlflow.search_runs('1')
display(runs)

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.percent change,metrics.length of predeiction,params.metric,params.file,params.length of predeiction,tags.mlflow.source.type,tags.mlflow.user,tags.mlflow.source.name
0,4de3bd06cfdd4bdf9e1c7c9e1b887ebc,1,FAILED,file:///root/finance_ml_analysis/aws/forecasti...,2021-04-21 03:23:29.434000+00:00,2021-04-21 03:23:29.692000+00:00,NaN,NaN,high,sagemaker-us-east-2-287085956542/stock_data_ra...,None,LOCAL,root,/opt/conda/lib/python3.7/site-packages/ipykern...
1,45496fc88a6649a4b07c4188b63abc36,1,FINISHED,file:///root/finance_ml_analysis/aws/forecasti...,2021-04-21 03:23:27.146000+00:00,2021-04-21 03:23:29.396000+00:00,0.023690,NaN,high,sagemaker-us-east-2-287085956542/stock_data_ra...,90,LOCAL,root,/opt/conda/lib/python3.7/site-packages/ipykern...
2,cc33a7ff7f1c45278773b71b139ef044,1,FINISHED,file:///root/finance_ml_analysis/aws/forecasti...,2021-04-21 03:23:24.907000+00:00,2021-04-21 03:23:27.111000+00:00,0.286511,NaN,high,sagemaker-us-east-2-287085956542/stock_data_ra...,90,LOCAL,root,/opt/conda/lib/python3.7/site-packages/ipykern...
3,7226d8bcdbb546229bedbd7797ceab2a,1,FINISHED,file:///root/finance_ml_analysis/aws/forecasti...,2021-04-21 03:23:21.732000+00:00,2021-04-21 03:23:24.869000+00:00,0.133838,NaN,high,sagemaker-us-east-2-287085956542/stock_data_ra...,90,LOCAL,root,/opt/conda/lib/python3.7/site-packages/ipykern...
4,d817ca1a95cc43c79090f5a18d0ab10d,1,FINISHED,file:///root/finance_ml_analysis/aws/forecasti...,2021-04-21 03:23:18.764000+00:00,2021-04-21 03:23:21.699000+00:00,0.109613,NaN,high,sagemaker-us-east-2-287085956542/stock_data_ra...,90,LOCAL,root,/opt/conda/lib/python3.7/site-packages/ipykern...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,7260406936a54ac5a070911f04c0b234,1,FINISHED,file:///root/finance_ml_analysis/aws/forecasti...,2021-04-21 03:09:45.436000+00:00,2021-04-21 03:09:47.556000+00:00,-0.476857,NaN,high,sagemaker-us-east-2-287085956542/stock_data_ra...,90,LOCAL,root,/opt/conda/lib/python3.7/site-packages/ipykern...
184,e74458627e9a48bab253e51aea4c87ee,1,FINISHED,file:///root/finance_ml_analysis/aws/forecasti...,2021-04-21 03:09:43.247000+00:00,2021-04-21 03:09:45.414000+00:00,0.952104,NaN,high,sagemaker-us-east-2-287085956542/stock_data_ra...,90,LOCAL,root,/opt/conda/lib/python3.7/site-packages/ipykern...
185,a9e5276e464e452a9dcb3aa50d4bf076,1,FINISHED,file:///root/finance_ml_analysis/aws/forecasti...,2021-04-21 03:09:41.125000+00:00,2021-04-21 03:09:43.225000+00:00,-0.011349,NaN,high,sagemaker-us-east-2-287085956542/stock_data_ra...,90,LOCAL,root,/opt/conda/lib/python3.7/site-packages/ipykern...
186,d9825791f7814c1a9fde435bf8405568,1,FINISHED,file:///root/finance_ml_analysis/aws/forecasti...,2021-04-21 03:09:35.063000+00:00,2021-04-21 03:09:41.102000+00:00,-0.550051,NaN,high,sagemaker-us-east-2-287085956542/stock_data_ra...,90,LOCAL,root,/opt/conda/lib/python3.7/site-packages/ipykern...


In [10]:
runs.sort_values('metrics.percent change', inplace=True, ascending=False)
display(runs.head(10))
files = runs.loc[:10, 'params.file']


,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.percent change,metrics.length of predeiction,params.metric,params.file,params.length of predeiction,tags.mlflow.source.type,tags.mlflow.user,tags.mlflow.source.name
13,66df3726ca2840a784283faf861dccf5,1,FINISHED,file:///root/finance_ml_analysis/aws/forecasti...,2021-04-21 03:22:41.562000+00:00,2021-04-21 03:22:43.690000+00:00,1.421840,NaN,high,sagemaker-us-east-2-287085956542/stock_data_ra...,90,LOCAL,root,/opt/conda/lib/python3.7/site-packages/ipykern...
10,23c36e48559d4dbcbd333cc47c6bed29,1,FINISHED,file:///root/finance_ml_analysis/aws/forecasti...,2021-04-21 03:23:01.543000+00:00,2021-04-21 03:23:05.089000+00:00,1.187934,NaN,high,sagemaker-us-east-2-287085956542/stock_data_ra...,90,LOCAL,root,/opt/conda/lib/python3.7/site-packages/ipykern...
163,5a4546e350e04bbdbb2a87f81865c154,1,FINISHED,file:///root/finance_ml_analysis/aws/forecasti...,2021-04-21 03:11:51.376000+00:00,2021-04-21 03:11:58.807000+00:00,1.034301,NaN,high,sagemaker-us-east-2-287085956542/stock_data_ra...,90,LOCAL,root,/opt/conda/lib/python3.7/site-packages/ipykern...
178,7159eedb552a4a3ab54d9d0a0bdc9493,1,FINISHED,file:///root/finance_ml_analysis/aws/forecasti...,2021-04-21 03:10:17.774000+00:00,2021-04-21 03:10:25.284000+00:00,1.034301,NaN,high,sagemaker-us-east-2-287085956542/stock_data_ra...,90,LOCAL,root,/opt/conda/lib/python3.7/site-packages/ipykern...
27,19cf17f535214a1c8fa3901d4e6a4582,1,FINISHED,file:///root/finance_ml_analysis/aws/forecasti...,2021-04-21 03:21:43.425000+00:00,2021-04-21 03:21:46.204000+00:00,0.964319,NaN,high,sagemaker-us-east-2-287085956542/stock_data_ra...,90,LOCAL,root,/opt/conda/lib/python3.7/site-packages/ipykern...
184,e74458627e9a48bab253e51aea4c87ee,1,FINISHED,file:///root/finance_ml_analysis/aws/forecasti...,2021-04-21 03:09:43.247000+00:00,2021-04-21 03:09:45.414000+00:00,0.952104,NaN,high,sagemaker-us-east-2-287085956542/stock_data_ra...,90,LOCAL,root,/opt/conda/lib/python3.7/site-packages/ipykern...
169,c8e9e58a3a444fb7b41f5f295f5ad91c,1,FINISHED,file:///root/finance_ml_analysis/aws/forecasti...,2021-04-21 03:11:16.854000+00:00,2021-04-21 03:11:18.947000+00:00,0.952104,NaN,high,sagemaker-us-east-2-287085956542/stock_data_ra...,90,LOCAL,root,/opt/conda/lib/python3.7/site-packages/ipykern...
138,36f824abd64d4ca3b22944f7f03a7401,1,FINISHED,file:///root/finance_ml_analysis/aws/forecasti...,2021-04-21 03:13:34.161000+00:00,2021-04-21 03:13:36.339000+00:00,0.949474,NaN,high,sagemaker-us-east-2-287085956542/stock_data_ra...,90,LOCAL,root,/opt/conda/lib/python3.7/site-packages/ipykern...
108,1f57794f2f6b437da55b84cf4417cae1,1,FINISHED,file:///root/finance_ml_analysis/aws/forecasti...,2021-04-21 03:15:56.946000+00:00,2021-04-21 03:16:00.409000+00:00,0.947481,NaN,high,sagemaker-us-east-2-287085956542/stock_data_ra...,90,LOCAL,root,/opt/conda/lib/python3.7/site-packages/ipykern...
50,a5e0ee08abe945229cd41657c4a5f60c,1,FINISHED,file:///root/finance_ml_analysis/aws/forecasti...,2021-04-21 03:20:19.100000+00:00,2021-04-21 03:20:21.177000+00:00,0.912662,NaN,high,sagemaker-us-east-2-287085956542/stock_data_ra...,90,LOCAL,root,/opt/conda/lib/python3.7/site-packages/ipykern...


In [ ]:

try:
    exp_id = mlflow.create_experiment('fbprophet_top')
except Exception as e:
    display(e)
    exp_id= mlflow.get_experiment_by_name('fbprophet_top')
for file in files:
    with mlflow.start_run(experiment_id=exp_id):
        mlflow.log_param('file', file)
        mlflow.log_param('metric',metric)
    #     mlflow.log_param('rolling_window', rolling_window)
        dft = pd.read_csv(f's3://{file}')#.dropna()
        df = dft[['Unnamed: 0', metric]]
        df.columns = ['ds', 'y']
        m = Prophet(growth='linear')
        df_cv = cross_validation(m, horizon = '90 days')
        df_p = performance_metrics(df_cv, rolling_window=rolling_window)
        mlflow.log_metric("rmse", df_p.loc[0,'RMSE'])
        mlflow.log_metric("MSE", df_p.loc[0,'MSE'])
        mlflow.log_metric("MAE", df_p.loc[0,'MAE'])
        mlflow.log_metric("MAPE", df_p.loc[0,'MAPE'])
        mlflow.log_metric("MDAPE", df_p.loc[0,'MDAPE'])
        mlflow.log_metric("SMAPE", df_p.loc[0,'SMAPE'])
        mlflow.log_metric("COVERAGE", df_p.loc[0,'COVERAGE'])   
results       
        

